In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# import tarfile
# train_tar = '/content/drive/MyDrive/Colab Notebooks/HDSC/Kaggle project/train-jpg.tar'
# dataset = tarfile.open('/content/drive/MyDrive/Colab Notebooks/HDSC/Kaggle project/train-jpg.tar')

In [ ]:
# for name in tqdm(dataset.getnames()):
#   print(name)
# dataset.extractall(path='/content/drive/MyDrive/Colab Notebooks/HDSC/Kaggle project/train img data')

In [ ]:
import pandas as pd
from tqdm import tqdm
from collections import Counter
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPool2D
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import fbeta_score
import tensorflow as tf

In [ ]:
train_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/HDSC/Kaggle project/train_v2.csv/train_v2.csv')
train_df.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [ ]:
train_df.tags.nunique()

449

In [ ]:
train_df.shape

(40479, 2)

In [ ]:
# train_df['list_tags'] = train_df['tags'].str.split(' ')
# row_tags = train_df['list_tags'].values
# tags_temp = [tag for row in row_tags for tag in row]
# counter_tags = Counter(tags_temp)
# df_tags = pd.DataFrame({'tag' : counter_tags.keys(), 
#                         'total' : counter_tags.values()}).sort_values('total')

In [ ]:
# all_tags = list(set(tags_temp))

In [ ]:
counts = {}
splitted_tags = train_df['tags'].map(lambda x: x.split(' '))
for labels in splitted_tags.values:
    for label in labels:
        counts[label] = counts[label] + 1  if label in counts else 0

In [ ]:
all_labels = splitted_tags.values
labels = list(set([y for x in all_labels for y in x]))

In [ ]:
# temp_dir = []
# for f, tags in train_df.values:
#   temp = path_train + f + '.jpg'
#   print(temp)

In [ ]:
filenames = train_df['image_name'].values
path_train = '/content/drive/MyDrive/Colab Notebooks/HDSC/Kaggle project/train img data/train-jpg/'
path_files = [os.path.join(path_train, filename + '.jpg') for filename in filenames]

In [ ]:
train_df

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road
...,...,...
40474,train_40474,clear primary
40475,train_40475,cloudy
40476,train_40476,agriculture clear primary
40477,train_40477,agriculture clear primary road


In [ ]:
df_train = train_df.sample(5000)

In [ ]:
resize = (128, 128)

In [ ]:
X = []
y = []

label_map = {l: i for i, l in enumerate(labels)}
inv_label_map = {i: l for l, i in label_map.items()}

for f, tags in tqdm(df_train.values):
  img = cv2.imread(os.path.join(path_train, f + '.jpg'))
  targets = np.zeros(17)
  for t in tags.split(' '):
    targets[label_map[t]] = 1 
    X.append(cv2.resize(img, resize))
    y.append(targets)

100%|██████████| 5000/5000 [1:15:22<00:00,  1.11it/s]


In [ ]:
%%time
X = np.array(X, np.float16) / 255.    
y = np.array(y, np.uint8)

CPU times: user 11.3 s, sys: 229 ms, total: 11.5 s
Wall time: 11.5 s


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [ ]:
tf.keras.backend.clear_session()
model = Sequential([
    Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same', input_shape=(128, 128, 3)),
    Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(64, kernel_size=(3, 3), activation='relu', padding='same'),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(0.2),

    Conv2D(128, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(128, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),

    Conv2D(256, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(256, kernel_size=(3, 3), activation='relu'),
    Conv2D(256, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),
    Dropout(0.2),

    Conv2D(512, kernel_size=(3, 3), activation='relu', padding='same'),
    Conv2D(512, kernel_size=(3, 3), activation='relu'),
    Conv2D(1024, kernel_size=(3, 3), activation='relu'),
    MaxPool2D(pool_size=(2, 2)),

    Flatten(),
    Dense(1024, activation='relu'),
    Dense(17, activation='sigmoid')])

In [ ]:
def fbeta_score_custom(y_true, y_pred):
    beta_squared = 4

    tp = K.sum(y_true * y_pred) + K.epsilon()
    fp = K.sum(y_pred) - tp
    fn = K.sum(y_true) - tp

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    result = (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())
    return result

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics= [fbeta_score_custom])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 64)      36928     
                                                                 
 conv2d_2 (Conv2D)           (None, 128, 128, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 64, 64, 64)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 64, 64, 128)       73856     
                                                        

In [ ]:
callback = tf.keras.callbacks.ModelCheckpoint('/content/drive/MyDrive/Colab Notebooks/HDSC/Kaggle project/model.h5', 
                                              save_best_only = True, 
                                              monitor = 'val_loss',
                                              verbose = 1)

In [ ]:
model_fit = model.fit(
    X_train, y_train,
    batch_size=128,
    epochs=10,
    verbose=1,
    callbacks=callback,
    validation_data=(X_val, y_val))

Epoch 1/10
91/91 [==============================] - ETA: 0s - loss: 0.3741 - fbeta_score_custom: 0.5418
Epoch 1: val_loss improved from inf to 0.26896, saving model to /content/drive/MyDrive/Colab Notebooks/HDSC/Kaggle project/model.h5
91/91 [==============================] - 81s 694ms/step - loss: 0.3741 - fbeta_score_custom: 0.5418 - val_loss: 0.2690 - val_fbeta_score_custom: 0.6003
Epoch 2/10
91/91 [==============================] - ETA: 0s - loss: 0.2602 - fbeta_score_custom: 0.6008
Epoch 2: val_loss improved from 0.26896 to 0.26374, saving model to /content/drive/MyDrive/Colab Notebooks/HDSC/Kaggle project/model.h5
91/91 [==============================] - 61s 670ms/step - loss: 0.2602 - fbeta_score_custom: 0.6008 - val_loss: 0.2637 - val_fbeta_score_custom: 0.5697
Epoch 3/10
91/91 [==============================] - ETA: 0s - loss: 0.2539 - fbeta_score_custom: 0.6059
Epoch 3: val_loss improved from 0.26374 to 0.25720, saving model to /content/drive/MyDrive/Colab Notebooks/HDSC/Kagg

In [ ]:
y_pred = model.predict(X_val, batch_size=128)
score = fbeta_score(y_val, np.array(y_pred) > 0.2, beta=2, average='samples')
print("F beta score: ", score)

23/23 [==============================] - 5s 201ms/step
F beta score:  0.8668724480623726


In [ ]:
# %%time
# import tarfile
# test_file = tarfile.open('/content/drive/MyDrive/Colab Notebooks/HDSC/Kaggle project/test-jpg.tar')
# test_file.extractall(path='/content/drive/MyDrive/Colab Notebooks/HDSC/Kaggle project/test img data')